In [19]:
import pandas as pd
from tqdm import tqdm
import os
import pickle
import cv2
from hand_tracker import HandTracker
import numpy as np

In [20]:
# 基础配置
root_dir = 'D:\\jester\\'
data_dir = 'D:\\jester\\data\\'
label_fn = 'jester-v1-labels.csv'
train_fn = 'jester-v1-train.csv'
test_fn = 'jester-v1-test.csv'
valid_fn = 'jester-v1-validation.csv'

train_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train.pkl'
valid_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid.pkl'

train_size = 10000
valid_size = 3000

joint_num = 21
joint_dim = 2

In [21]:
# 模型相关初始化
palm_model_path = ".\\models\\palm_detection.tflite"
landmark_model_path = ".\\models\\hand_landmark.tflite"
anchors_path = ".\\data\\anchors.csv" 
# box_shift determines 
from hand_tracker import HandTracker
detector = HandTracker(palm_model_path, landmark_model_path, anchors_path,
                       box_shift=0.2, box_enlarge=1.3)
hand_pres_thresh = 1e-7 # 手部landmark评分的丢弃阈值

palm tensor input 信息 [{'name': 'input_1', 'index': 234, 'shape': array([  1, 256, 256,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
palm tensor ouput 信息 [{'name': 'regressors/concat', 'index': 295, 'shape': array([   1, 2944,   18]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'classificators/concat', 'index': 14, 'shape': array([   1, 2944,    1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
landmark tensor input 信息 [{'name': 'input_1', 'index': 0, 'shape': array([  1, 256, 256,   3]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
landmark tensor ouput 信 息 [{'name': 'ld_21_2d', 'index': 893, 'shape': array([ 1, 42]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}, {'name': 'output_handflag', 'index': 894, 'shape': array([1, 1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
hand tracker built ok


In [22]:
# 加载动作类别
cls_csv = pd.read_csv(root_dir + label_fn, sep=';')
cls_dict = dict(zip(cls_csv.clsid, cls_csv.clsname))
cls_dict_verse = dict(zip(cls_csv.clsname, cls_csv.clsid))
# print(cls_dict[1], cls_dict_verse['Swiping Left'])

In [23]:
print(cls_dict)

{1: 'Swiping Left', 2: 'Swiping Right', 3: 'Swiping Down', 4: 'Swiping Up', 5: 'Pushing Hand Away', 6: 'Pulling Hand In', 7: 'Sliding Two Fingers Left', 8: 'Sliding Two Fingers Right', 9: 'Sliding Two Fingers Down', 10: 'Sliding Two Fingers Up', 11: 'Pushing Two Fingers Away', 12: 'Pulling Two Fingers In', 13: 'Rolling Hand Forward', 14: 'Rolling Hand Backward', 15: 'Turning Hand Clockwise', 16: 'Turning Hand Counterclockwise', 17: 'Zooming In With Full Hand', 18: 'Zooming Out With Full Hand', 19: 'Zooming In With Two Fingers', 20: 'Zooming Out With Two Fingers', 21: 'Thumb Up', 22: 'Thumb Down', 23: 'Shaking Hand', 24: 'Stop Sign', 25: 'Drumming Fingers', 26: 'No gesture', 27: 'Doing other things'}


## 加载训练集

In [24]:
try:
    Train = pickle.load(open(train_output_fn, "rb")) # 先加载一波旧数据
except:
    Train = {}
    Train['pose'] = []
    Train['label'] = []
    
print(len(Train['label']))

7486


In [25]:
# 加载训练集
train_data = pd.read_csv(root_dir + train_fn, sep=';')
train_dict = dict(zip(train_data.id, train_data.clsid))

save_count = 0

for index in tqdm(list(train_dict.items())[len(Train['label']):train_size]):
    image_path = data_dir + str(index[0])
    image_file_list = os.listdir(image_path)    
    # 逐个加载并用 HandTracker 得出骨架数据
    skeleton_list = np.empty((len(image_file_list),(joint_dim * joint_num)), dtype = float, order = 'C')
    for file_index in range(len(image_file_list)):
        file = image_file_list[file_index]
        image = cv2.imread(image_path + '\\' + file)[:,:,::-1]
        image = cv2.resize(image,(256,256)) # resize 为 mediapipe 处理的格式，且实时检测也要保持这个
        kp, box, confidence = detector(image)
        if kp is None or confidence < hand_pres_thresh:        
            # 暂时按0处理
            kp = np.zeros(joint_dim * joint_num)
        else:
            kp = kp.reshape(joint_dim * joint_num)
        skeleton_list[file_index] = kp
    Train['pose'].append(skeleton_list)
    Train['label'].append(index[1])

    save_count += 1
    if save_count % 50 == 0:
        pickle.dump(Train, open(train_output_fn, "wb"))

# TODO: 空骨架 --> 开头(0,0,0)，中间插值
# TODO: 中值滤波


  3%|██                                                                            | 67/2514 [16:01<9:45:12, 14.35s/it]


  5%|████▏                                                                        | 135/2514 [32:10<9:27:01, 14.30s/it]


  8%|██████▏                                                                      | 203/2514 [48:32<9:12:38, 14.35s/it]


 11%|████████                                                                   | 271/2514 [1:04:17<8:52:03, 14.23s/it]


 13%|██████████                                                                 | 339/2514 [1:20:18<8:35:15, 14.21s/it]


...

 35%|██████████████████████████▎                                                | 883/2514 [3:42:44<6:51:25, 15.14s/it]


 38%|████████████████████████████▎                                              | 951/2514 [3:59:39<6:33:53, 15.12s/it]


 41%|█████████████████████████████▉                                            | 1019/2514 [4:16:11<6:15:52, 15.09s/it]


 42%|███████████████████████████████▎                                          | 1064/2514 [4:27:16<6:04:14, 15.07s/it]

KeyboardInterrupt: 

In [26]:
# Train数据保存到pickle文件
print('已保存训练数据:', len(Train['label']))
pickle.dump(Train, open(train_output_fn, "wb"))

已保存训练数据: 8550


## 加载验证集

In [27]:
try:
    Valid = pickle.load(open(valid_output_fn, "rb")) # 先加载一波旧数据
except:
    Valid = {}
    Valid['pose'] = []
    Valid['label'] = []
    
print(len(Valid['label']))

1561


In [ ]:
# 加载验证集

valid_data = pd.read_csv(root_dir + valid_fn, sep=';')
valid_dict = dict(zip(valid_data.id, valid_data.clsid))

save_count = 0

for index in tqdm(list(valid_dict.items())[len(Valid['label']):valid_size]):
    image_path = data_dir + str(index[0])
    image_file_list = os.listdir(image_path)    
    # 逐个加载并用 HandTracker 得出骨架数据
    skeleton_list = np.empty((len(image_file_list),(joint_dim * joint_num)), dtype = float, order = 'C')
    for file_index in range(len(image_file_list)):
        file = image_file_list[file_index]
        image = cv2.imread(image_path + '\\' + file)[:,:,::-1]
        image = cv2.resize(image,(256,256)) # resize 为 mediapipe 处理的格式，且实时检测也要保持这个
        kp, box, confidence = detector(image)
        if kp is None or confidence < hand_pres_thresh:     
            # 暂时按0处理
            kp = np.zeros(joint_dim * joint_num)
        else:
            kp = kp.reshape(joint_dim * joint_num)
        skeleton_list[file_index] = kp
    Valid['pose'].append(skeleton_list)
    Valid['label'].append(index[1])

    save_count += 1
    if save_count % 50 == 0:
        pickle.dump(Valid, open(valid_output_fn, "wb"))


# TODO: 空骨架 --> 开头(0,0,0)，中间插值
# TODO: 中值滤波

In [28]:
# Valid数据保存到pickle文件
print('已保存验证数据:', len(Valid['label']))
pickle.dump(Valid, open(valid_output_fn, "wb"))

已保存验证数据: 1561


## 删除空frame的数据

In [29]:

# 训练集
train_noempty_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train_noempty.pkl'
Train_NoEmpty = pickle.load(open(train_output_fn, "rb")) # 先加载一波旧数据
poses = Train_NoEmpty['pose']

for pose_index in tqdm(range(len(poses))):
    frame_index = 0
    while frame_index < len(poses[pose_index]):
        if poses[pose_index][frame_index][0] == 0:
            poses[pose_index] = np.delete(poses[pose_index], frame_index, axis=0)
        else:
            frame_index += 1
    if len(poses[pose_index]) == 0:
        poses[pose_index] = np.zeros((1,joint_num*joint_dim))
    pose = np.zeros((1,joint_num*joint_dim))
    
print('已保存(无空帧)训练数据:', len(Train_NoEmpty['label']))
pickle.dump(Train_NoEmpty, open(train_noempty_output_fn, "wb"))



  0%|                                                                                         | 0/8550 [00:00<?, ?it/s]

  3%|██                                                                           | 233/8550 [00:00<00:03, 2268.20it/s]

  5%|███▊                                                                         | 430/8550 [00:00<00:03, 2092.96it/s]

  7%|█████▍                                                                       | 597/8550 [00:00<00:04, 1962.62it/s]

  9%|███████▎                                                                     | 806/8550 [00:00<00:03, 1990.53it/s]

 12%|████████▉                                                                   | 1007/8550 [00:00<00:03, 1991.51it/s]

 14%|██████████▉                                                                 | 1225/8550 [00:00<00:03, 2020.18it/s]

 16%|████████████▍                                                               | 1406/8550 [00:00<00:03, 1974.45it/s]

 19%|██████████████▋          

已保存(无空帧)训练数据: 8550


In [30]:
# 验证集
valid_noempty_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid_noempty.pkl'
Valid_NoEmpty = pickle.load(open(valid_output_fn, "rb")) # 先加载一波旧数据
poses = Valid_NoEmpty['pose']

for pose_index in tqdm(range(len(poses))):
    frame_index = 0
    while frame_index < len(poses[pose_index]):
        if poses[pose_index][frame_index][0] == 0:
            poses[pose_index] = np.delete(poses[pose_index], frame_index, axis=0)
        else:
            frame_index += 1
    if len(poses[pose_index]) == 0:
        poses[pose_index] = np.zeros((1,joint_num*joint_dim))
    pose = np.zeros((1,joint_num*joint_dim))
    
print('已保存(无空帧)验证数据:', len(Valid_NoEmpty['label']))
pickle.dump(Valid_NoEmpty, open(valid_noempty_output_fn, "wb"))



  0%|                                                                                         | 0/1561 [00:00<?, ?it/s]

 12%|█████████▎                                                                   | 189/1561 [00:00<00:00, 1822.18it/s]

 20%|███████████████▎                                                             | 310/1561 [00:00<00:00, 1501.60it/s]

 31%|███████████████████████▌                                                     | 478/1561 [00:00<00:00, 1561.18it/s]

 42%|████████████████████████████████▍                                            | 657/1561 [00:00<00:00, 1614.60it/s]

 50%|██████████████████████████████████████▌                                      | 782/1561 [00:00<00:00, 1531.43it/s]

 63%|████████████████████████████████████████████████▏                            | 977/1561 [00:00<00:00, 1598.07it/s]

 76%|█████████████████████████████████████████████████████████▋                  | 1185/1561 [00:00<00:00, 1664.10it/s]

 91%|█████████████████████████

已保存(无空帧)验证数据: 1561


## 1-6 class 删除空骨架帧 的数据

In [ ]:
# 训练集
train_1to6_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\train_1to6.pkl'
Train_1to6 = pickle.load(open(train_noempty_output_fn, "rb")) # 先加载一波旧数据
poses = Train_1to6['pose']
labels = Train_1to6['label']
pose_index=0
max_cls = 6

while pose_index < len(labels):
    if labels[pose_index] <= max_cls:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)

print('已保存(class 1-6)训练数据:', len(Train_1to6['label']))
pickle.dump(Train_1to6, open(train_1to6_output_fn, "wb"))

In [ ]:
# 验证集
valid_1to6_output_fn = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\valid_1to6.pkl'
Valid_1to6 = pickle.load(open(valid_noempty_output_fn, "rb")) # 先加载一波旧数据
poses = Valid_1to6['pose']
labels = Valid_1to6['label']
pose_index=0
max_cls = 6

while pose_index < len(labels):
    if labels[pose_index] <= max_cls:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)

print('已保存(class 1-6)验证数据:', len(Valid_1to6['label']))
pickle.dump(Valid_1to6, open(valid_1to6_output_fn, "wb"))

## 分类数 [1, N]；非空帧数 [K, ) 的数据

In [43]:
thresh_frame_num = 15 # 非空帧数 (0即所有)
thresh_frame_rate = 0.3 # 非空帧占比 (0即所有)
max_cls = 12 # class最大数
rootdir = 'C:\\Users\\zys98\\Desktop\\graduation\\github-repo\\sk_dd_2019\\codes\\data\\JESTER\\'

count = 0
for i in range(len(Train_NoEmpty['pose'])):
    data = Train_NoEmpty['pose'][i]
    odata = Train['pose'][i]
#     print(i,data.shape[0])
    if data.shape[0]/odata.shape[0] >= thresh_frame_rate:
        count += 1

print('非空阈值样本占比', count/len(Train_NoEmpty['pose']))


# 训练集
train_noempty_fn = 'train_noempty.pkl'
train_gt20_1to6_fn = 'train_gt20_1to6.pkl'
Train_GT20_1to6 = pickle.load(open(rootdir + train_noempty_fn, "rb")) # 先加载一波旧数据
poses = Train_GT20_1to6['pose']
oposes = Train['pose']
labels = Train_GT20_1to6['label']
pose_index=0

while pose_index < len(labels):
    if labels[pose_index] <= max_cls and poses[pose_index].shape[0] >= thresh_frame_num and poses[pose_index].shape[0]/oposes[pose_index].shape[0] >= thresh_frame_rate:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)
    
print('已保存(1-'+str(max_cls)+' class 非空帧数 > '+str(thresh_frame_num)+' 的数据)训练数据:', len(Train_GT20_1to6['label']))
pickle.dump(Train_GT20_1to6, open(rootdir + train_gt20_1to6_fn, "wb"))

# 验证集
valid_noempty_fn = 'valid_noempty.pkl'
valid_gt20_1to6_fn = 'valid_gt20_1to6.pkl'
Valid_GT20_1to6 = pickle.load(open(rootdir + valid_noempty_fn, "rb")) # 先加载一波旧数据
poses = Valid_GT20_1to6['pose']
oposes = Valid['pose']
labels = Valid_GT20_1to6['label']
pose_index=0

while pose_index < len(labels):
    if labels[pose_index] <= max_cls and poses[pose_index].shape[0] >= thresh_frame_num  and poses[pose_index].shape[0]/oposes[pose_index].shape[0] >= thresh_frame_rate:
        pose_index += 1
    else:
        poses.pop(pose_index)
        labels.pop(pose_index)
    
print('已保存(1-'+str(max_cls)+' class 非空帧数 > '+str(thresh_frame_num)+' 的数据)验证数据:', len(Valid_GT20_1to6['label']))
pickle.dump(Valid_GT20_1to6, open(rootdir + valid_gt20_1to6_fn, "wb"))

非空阈值样本占比 0.20514619883040935
已保存(1-12 class 非空帧数 > 15 的数据)训练数据: 258
已保存(1-12 class 非空帧数 > 15 的数据)验证数据: 42


## 用于统计各数据集空骨架的概率

In [ ]:
frame_count = 0
empty_count = 0
all_emtpy = 0
for pose in Train_1to6['pose']:
    this_frame = 0
    this_empty = 0
    for frame in pose:
        this_frame += 1
        frame_count += 1
        if frame[0] == 0:
            empty_count += 1
            this_empty += 1
    if this_empty == this_frame:
        all_emtpy += 1
print('总空帧率(%):',empty_count / frame_count * 100)
print('空样本率(%)',all_emtpy / len(Train_1to6['pose']) * 100)